# RunPod RTX 3090 Setup Verification

This notebook verifies your RunPod environment is properly configured for the poisoning attack analysis experiments.

## Hardware Specs
- **GPU**: RTX 3090 (24GB VRAM)
- **CPU**: 32 cores
- **RAM**: 125GB

Run all cells to verify your setup.

## 1. Check GPU Availability

In [9]:
import torch
import torch.cuda as cuda

print("="*60)
print("GPU INFORMATION")
print("="*60)
print(f"CUDA Available: {cuda.is_available()}")

if cuda.is_available():
    print(f"GPU Name: {cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Count: {cuda.device_count()}")
    print(f"Current GPU: {cuda.current_device()}")
    
    # Memory info
    mem_total = cuda.get_device_properties(0).total_memory / 1e9
    mem_allocated = cuda.memory_allocated(0) / 1e9
    mem_reserved = cuda.memory_reserved(0) / 1e9
    
    print(f"\nMemory Info:")
    print(f"  Total VRAM: {mem_total:.2f} GB")
    print(f"  Allocated: {mem_allocated:.2f} GB")
    print(f"  Reserved: {mem_reserved:.2f} GB")
    print(f"  Free: {mem_total - mem_reserved:.2f} GB")
    
    # Check if it's RTX 2000
    if "2000" in cuda.get_device_name(0):
        print("\n✅ RTX 2000 detected!")
    else:
        print(f"\n⚠️ Warning: Expected RTX 3090, but found {cuda.get_device_name(0)}")
else:
    print("❌ CUDA is not available! PyTorch will run on CPU.")
    print("Please check your CUDA installation.")

GPU INFORMATION
CUDA Available: True
GPU Name: NVIDIA RTX 2000 Ada Generation
CUDA Version: 12.8
GPU Count: 3
Current GPU: 0

Memory Info:
  Total VRAM: 16.73 GB
  Allocated: 0.00 GB
  Reserved: 0.00 GB
  Free: 16.73 GB

✅ RTX 2000 detected!


## 2. Check CPU and RAM

In [10]:
import multiprocessing
import psutil
import os

print("="*60)
print("CPU & RAM INFORMATION")
print("="*60)

# CPU info
cpu_count = multiprocessing.cpu_count()
print(f"CPU Cores: {cpu_count}")

# RAM info
ram_total = psutil.virtual_memory().total / 1e9
ram_available = psutil.virtual_memory().available / 1e9
ram_percent = psutil.virtual_memory().percent

print(f"\nRAM Info:")
print(f"  Total RAM: {ram_total:.2f} GB")
print(f"  Available RAM: {ram_available:.2f} GB")
print(f"  Used: {ram_percent:.1f}%")

# Verify specs
if cpu_count >= 32:
    print(f"\n✅ CPU cores: {cpu_count} (expected ~32)")
else:
    print(f"\n⚠️ CPU cores: {cpu_count} (expected 32)")

if ram_total >= 120:
    print(f"✅ RAM: {ram_total:.0f} GB (expected ~125 GB)")
else:
    print(f"⚠️ RAM: {ram_total:.0f} GB (expected 125 GB)")

CPU & RAM INFORMATION
CPU Cores: 48

RAM Info:
  Total RAM: 270.09 GB
  Available RAM: 242.92 GB
  Used: 10.1%

✅ CPU cores: 48 (expected ~32)
✅ RAM: 270 GB (expected ~125 GB)


## 3. Test PyTorch Dependencies

In [11]:
!pip install pandas matplotlib seaborn

In [12]:
print("="*60)
print("DEPENDENCY CHECK")
print("="*60)

dependencies = [
    'torch',
    'torchvision',
    'numpy',
    'pandas',
    'matplotlib',
    'seaborn',
    'yaml'
]

import importlib

for dep in dependencies:
    try:
        if dep == 'yaml':
            module = importlib.import_module(dep)
        else:
            module = importlib.import_module(dep)
        version = getattr(module, '__version__', 'unknown')
        print(f"✅ {dep:15s} version: {version}")
    except ImportError:
        print(f"❌ {dep:15s} NOT INSTALLED")

DEPENDENCY CHECK
✅ torch           version: 2.8.0+cu128
✅ torchvision     version: 0.23.0+cu128
✅ numpy           version: 2.1.2
✅ pandas          version: 2.3.3
✅ matplotlib      version: 3.10.7
✅ seaborn         version: 0.13.2
✅ yaml            version: 6.0.3


## 4. Test Project Imports

In [13]:
import sys
import os

# Add project directory to path (adjust if needed)
project_path = '/workspace/poisoning_attack_analysis'  # Change this to your project path
if os.path.exists(project_path):
    sys.path.insert(0, project_path)
    print(f"✅ Project path added: {project_path}")
else:
    print(f"⚠️ Project path not found: {project_path}")
    print("Please update the project_path variable above.")

print("\n" + "="*60)
print("PROJECT MODULE CHECK")
print("="*60)

try:
    from models import ScalableCNN
    print("✅ models.py imported successfully")
except ImportError as e:
    print(f"❌ Failed to import models.py: {e}")

try:
    from data_utils import load_global_dataset, partition_data_dirichlet
    print("✅ data_utils.py imported successfully")
except ImportError as e:
    print(f"❌ Failed to import data_utils.py: {e}")

try:
    from utils import train_client, evaluate_model, fed_avg
    print("✅ utils.py imported successfully")
except ImportError as e:
    print(f"❌ Failed to import utils.py: {e}")

✅ Project path added: /workspace/poisoning_attack_analysis

PROJECT MODULE CHECK
✅ models.py imported successfully
✅ data_utils.py imported successfully
✅ utils.py imported successfully


## 5. Quick GPU Test (Training a Small Model)

In [14]:
import torch
import torch.nn as nn
import time

print("="*60)
print("GPU TRAINING TEST")
print("="*60)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Create a simple model
model = nn.Sequential(
    nn.Linear(1000, 5000),
    nn.ReLU(),
    nn.Linear(5000, 5000),
    nn.ReLU(),
    nn.Linear(5000, 10)
).to(device)

# Create dummy data
x = torch.randn(128, 1000).to(device)
y = torch.randint(0, 10, (128,)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Training loop
print("\nRunning 100 training iterations...")
start_time = time.time()

for i in range(100):
    optimizer.zero_grad()
    outputs = model(x)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    
    if (i + 1) % 20 == 0:
        print(f"  Iteration {i+1}/100, Loss: {loss.item():.4f}")

elapsed = time.time() - start_time
print(f"\n✅ Training completed in {elapsed:.2f} seconds")
print(f"   Average: {elapsed/100*1000:.2f} ms per iteration")

if torch.cuda.is_available():
    print(f"\nGPU Memory Usage:")
    print(f"  Allocated: {torch.cuda.memory_allocated(0)/1e9:.2f} GB")
    print(f"  Reserved: {torch.cuda.memory_reserved(0)/1e9:.2f} GB")

GPU TRAINING TEST
Using device: cuda

Running 100 training iterations...
  Iteration 20/100, Loss: 2.0146
  Iteration 40/100, Loss: 1.7329
  Iteration 60/100, Loss: 1.4475
  Iteration 80/100, Loss: 1.1622
  Iteration 100/100, Loss: 0.8946

✅ Training completed in 0.67 seconds
   Average: 6.75 ms per iteration

GPU Memory Usage:
  Allocated: 0.26 GB
  Reserved: 0.29 GB


## 6. Configuration File Check

In [15]:
import yaml
import os

print("="*60)
print("CONFIG FILE CHECK")
print("="*60)

config_dir = os.path.join(project_path, 'configs')
config_files = [
    'config_exp0_mnist.yaml',
    'config_exp0_cifar10.yaml',
    'config_exp1_mnist.yaml',
    'config_exp1_cifar10.yaml',
]

for config_file in config_files:
    config_path = os.path.join(config_dir, config_file)
    if os.path.exists(config_path):
        try:
            with open(config_path, 'r') as f:
                config = yaml.safe_load(f)
            batch_size = config['defaults']['batch_size']
            num_workers = config['defaults'].get('num_parallel_workers', 'not set')
            print(f"✅ {config_file:30s} batch_size={batch_size}, workers={num_workers}")
        except Exception as e:
            print(f"❌ {config_file:30s} Error: {e}")
    else:
        print(f"⚠️ {config_file:30s} NOT FOUND")

CONFIG FILE CHECK
✅ config_exp0_mnist.yaml         batch_size=128, workers=1
✅ config_exp0_cifar10.yaml       batch_size=128, workers=1
✅ config_exp1_mnist.yaml         batch_size=128, workers=1
✅ config_exp1_cifar10.yaml       batch_size=128, workers=1


## 7. Summary

In [17]:
print("="*60)
print("SETUP SUMMARY")
print("="*60)

checks = {
    "GPU (CUDA)": torch.cuda.is_available(),
    "RTX 2000": "2000" in torch.cuda.get_device_name(0) if torch.cuda.is_available() else False,
    "CPU Cores (≥32)": multiprocessing.cpu_count() >= 32,
    "RAM (≥120 GB)": psutil.virtual_memory().total / 1e9 >= 120,
}

for check_name, status in checks.items():
    status_icon = "✅" if status else "❌"
    print(f"{status_icon} {check_name}")

if all(checks.values()):
    print("\n" + "="*60)
    print("🎉 ALL CHECKS PASSED! You're ready to run experiments!")
    print("="*60)
    print("\nNext steps:")
    print("1. Navigate to your project directory")
    print("2. Run a test experiment:")
    print("   python experiment_runner.py configs/config_exp1_mnist.yaml")
else:
    print("\n⚠️ Some checks failed. Please review the output above.")

SETUP SUMMARY
✅ GPU (CUDA)
✅ RTX 2000
✅ CPU Cores (≥32)
✅ RAM (≥120 GB)

🎉 ALL CHECKS PASSED! You're ready to run experiments!

Next steps:
1. Navigate to your project directory
2. Run a test experiment:
   python experiment_runner.py configs/config_exp1_mnist.yaml
